<a href="https://colab.research.google.com/github/gytjd/QA_Data_Augmentation/blob/main/quantuam_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# 이미지 전처리

import cv2
import numpy as np
from scipy import ndimage
from statistics import median

from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME='Quantuam_AI'

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))


def process_image(img_path, save_path):
    # 이미지 로딩
    img = cv2.imread(img_path)

    # 1. 이미지를 grayscale로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. 이미지 노이즈 제거
    denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)

    # 3. 가우시안 블러링 실행
    blurred = cv2.GaussianBlur(denoised, (5,5), 0)

    # 4. 이미지의 엣지검출
    edges = cv2.Canny(blurred, 50, 150)

    # 5. 확률적 허프변환에 의한 선분 검출
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=5)

    # 6. 선의 유무에 따라
    if lines is not None:
        angles = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # 6-2-1. 추출된 선들 기반으로 각도 계산
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            angles.append(angle)

        # 6-2-2. 각도 계산에 사용된 선의 길이가 선들의 중앙값보다 작은 각도 제외
        med_angle = median(angles)
        filtered_angles = [angle for angle in angles if abs(angle) > med_angle]

        # 6-2-3. 남은 각도의 분포 기반으로 수정각도 계산
        correct_angle = np.mean(filtered_angles)

        # 6-2-4. 수정각도가 0 또는 90이 아닐때 수정각도만큼 이미지 회전
        if correct_angle not in [0, 90]:
            rotated = ndimage.rotate(img, correct_angle)
        else:
            rotated = img.copy()
    else:
        # 6-1. 각도 계산이 불가능하므로 해당 이미지대로 저장
        rotated = img.copy()

    # 7. 학습을 위해 이미지의 가로길이, 세로길이, DPI를 일관되게 저장
    resized = cv2.resize(rotated, (300, 300))
    cv2.imwrite("save_path.jpg", resized)
    return resized



def aug_Spin(image, rotation_range=(-360, 360)):
    # 1. 랜덤 회전
    angle = np.random.randint(rotation_range[0], rotation_range[1])
    rotated = ndimage.rotate(image, angle, reshape=False, mode='nearest')
    return rotated



def aug_Bright(image, brightness_range=(-300, 300)):
    # 3. 랜덤 밝기 조절
    brightness = np.random.randint(brightness_range[0], brightness_range[1])
    brightened = cv2.convertScaleAbs(image, alpha=1, beta=brightness)
    return brightened

def aug_Constrast(image, contrast_range=(0.0, 5.0)):
    # 4. 랜덤 대비 조절
    contrast = np.random.uniform(contrast_range[0], contrast_range[1])
    contrasted = cv2.convertScaleAbs(image, alpha=contrast, beta=0)
    return contrasted

def aug_Noise(image, noise_stddev=50):

    # 6. 랜덤 노이즈 추가
    mean = 0
    stddev = noise_stddev
    noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
    noisy = cv2.add(image, noise)

    return noisy

def aug_Blur(image, blur_kernel=(5, 5)):
    # 7. 이미지 블러 처리
    blurred = cv2.GaussianBlur(image, blur_kernel, 10)
    return blurred


def aug_Shift(image, shift_range=(0, 100)):
    # 8. 이미지 이동
    h, w, _ = image.shape
    shift_x = np.random.randint(shift_range[0], shift_range[1])
    shift_y = np.random.randint(shift_range[0], shift_range[1])
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    shifted = cv2.warpAffine(image, M, (w, h))
    return shifted

def aug_ColorShift(image, shift_range=(-200, 200)):
    # 11. 이미지의 각 채널을 랜덤하게 색조 변경
    shifted = image.copy()
    for c in range(3):  # RGB 채널에 대해 반복
        shift_value = np.random.randint(shift_range[0], shift_range[1])
        shifted[:, :, c] = np.clip(shifted[:, :, c] + shift_value, 0, 255)
    return shifted

def aug_Grayscale(image):
    # 12. 이미지를 흑백으로 변환
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(grayscale, cv2.COLOR_GRAY2BGR)


def save_augmented_images(original_path, save_dir, num_augments=20):
    original = process_image(original_path, None)

    for i in range(num_augments):
      temp=aug_Spin(original)
      if i%8==0:
        augmented = aug_Bright(temp)
      elif i%8==1:
        augmented = aug_Constrast(temp)
      elif i%8==2 or i%8==3:
        augmented = aug_Noise(temp)
      elif i%8==4:
        augmented = aug_Blur(temp)
      elif i%8==5:
        augmented = aug_Shift(temp)
      elif i%8==6:
        augmented = aug_ColorShift(temp)
      elif i%8== 7:
        augmented = aug_Grayscale(temp)

      save_path = f"{save_dir}/augmented_{i}_{original_path.split('/')[-1]}"
      cv2.imwrite(save_path, augmented)


# 이미지 경로와 저장 디렉토리를 지정하여 함수 호출
save_augmented_images('/content/data_input/1.jpeg', '/content/drive/MyDrive/Quantuam_AI')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
